In [4]:
import os
import sys
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
def evaluate_student_answer(question, student_answer, model_answer):
    # Değerlendirme prompt'u
    prompt = f"""
    Soru: {question}
    Öğrenci Cevabı: {student_answer}
    Model Cevap: {model_answer}
    
    Lütfen öğrencinin cevabını aşağıdaki kriterlere göre değerlendirin:
    1. Doğruluk (0-10)
    2. Açıklama Kalitesi (0-10)
    3. Eksiklikler (Varsa belirtin)
    4. Genel Puan (0-10)
    """
    
    # Tokenize ve model çıkışı üret
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        do_sample=True
    )
    
    # Çıktıyı metne dönüştür
    evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return evaluation

# Örnek Kullanım
result = evaluate_student_answer(
    question="Python'da bir listeyi nasıl ters çevirirsiniz?",
    student_answer="reverse() metodu ile ters çeviririm.",
    model_answer="list.reverse() metodu veya [::-1] dilimleme işlemi kullanılır."
)
print("Değerlendirme Sonucu:\n", result)

Değerlendirme Sonucu:
 
    Soru: Python'da bir listeyi nasıl ters çevirirsiniz?
    Öğrenci Cevabı: reverse() metodu ile ters çeviririm.
    Model Cevap: list.reverse() metodu veya [::-1] dilimleme işlemi kullanılır.
    
    Lütfen öğrencinin cevabını aşağıdaki kriterlere göre değerlendirin:
    1. Doğruluk (0-10)
    2. Açıklama Kalitesi (0-10)
    3. Eksiklikler (Varsa belirtin)
    4. Genel Puan (0-10)
     """

    dogruluk = 8
    aciklama_kalitesi = 7
    eksiklikler = "Öğrencinin cevabı tamamen doğru ancak, reverse() metodunun dönen değeri yoktur. Liste üzerinde işlem yapılır ve liste kendisi değişir. Ayrıca, [::-1] yöntemini de belirtmek daha kapsamlı olur."
    genel_puan = 8

    return dogruluk, aciklama_kalitesi, eksiklikler, genel_puan


# Test
dogruluk, aciklama_kalitesi, eksiklikler, genel_puan = evaluate_student_answer(student_answer)
print(f"Doğruluk: {dogruluk}")
print(f"Açıklama Kalitesi: {aciklama_kalitesi}")
print(f"Eksiklikler: {eksiklikler}")
print(f"Genel Puan